In [9]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

demand = [[6048,5343,3240,4598,8172,6341,3834,2619,2128,2943], [873,1123,1106,678,858,939,1060,730,826,1471],[465,1376,636,514,1083,640,977,1022,938,879],[
183,156,48,27,159,105,122,62,102,19]]

majorsetupcost =300
setupcost = 50
holdingcost = [0.001, 0.01, 0.01, 0.1]

m = gp.Model("JRP")

# Create variables
lotsize = {}
inventory ={}
setup = {}
m_setup = {}
for t in range(10):
  m_setup[t] = m.addVar(vtype=GRB.BINARY,obj=majorsetupcost)  
  for k in range(4):  
   setup[k,t] = m.addVar(vtype=GRB.BINARY, obj=setupcost)     
   lotsize[k,t] = m.addVar()
   inventory[k,t] = m.addVar(obj=holdingcost[k])

# inventory balance
for k in range(4):
  m.addConstr(inventory[k,0] == lotsize[k,0]-demand[k][0])
  for t in range(1,10):
     m.addConstr(inventory[k,t-1]+lotsize[k,t]-demand[k][t] == inventory[k,t])
    
# Logic constraints
for t in range(10):
  for k in range(4):  
    m.addConstr(setup[k,t] <= m_setup[t])
    m.addConstr(lotsize[k,t] <= 100000*setup[k,t])

m.optimize()

print("Objective: "+str(m.objVal))
for v in m.getVars():
  print(v.x)


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 120 rows, 130 columns and 276 nonzeros
Model fingerprint: 0x3024e4a3
Variable types: 80 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-03, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 8e+03]
Found heuristic solution: objective 3982.2570000
Presolve removed 16 rows and 17 columns
Presolve time: 0.00s
Presolved: 104 rows, 113 columns, 240 nonzeros
Variable types: 68 continuous, 45 integer (45 binary)

Root relaxation: objective 7.176665e+02, 53 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  717.66650    0   35 3982.25700  717.66650  82.0%     -    0s
H    0     0                    3924.0340000  717.66650  81.7%     -    0s
H    0     0                    1889.2180000  717.66650  